In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import geopy
from time import sleep
from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pycountry
import os
import difflib
import re


# hide warnings
import warnings
warnings.filterwarnings('ignore')

#os.getcwd() - code to get current working directory
# import requests
# import urllib

In [2]:
def do_geocode(address):
    """
    create geopy class instance
    and path address to get latitude, longitude
    """
    geolocator = Nominatim(user_agent="my_app")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    try:
        lat = geocode(address).latitude
        long = geocode(address).longitude
        return lat, long
    except GeocoderTimedOut:
        return do_geocode(address)

def get_country_code(country):
    """
    create pycountry instance and pass country name to get country code
    """
    mapping = {country.name: country.alpha_2 for country in pycountry.countries}
    return mapping.get(country)

In [5]:
#base_dir - path to root directory
#realms_file - string name of file with realms data (i.e. "realms.csv")
#new_file - string name for new file that will be created (i.e. "clean.csv")

def clean_data(base_dir, realms_file, new_file):
    """
    function to fetch, clean and save COVID-19 data
    """
    
    url = "https://raw.githubusercontent.com/datasets/covid-19/master/data/time-series-19-covid-combined.csv"

    # read data into pandas df and do some renaming&cleaning
    data = pd.read_csv(url, error_bad_lines=False, parse_dates=['Date'])
    data = data.rename(columns={'Country/Region': 'Country'})
    data = data[data.Country != 'Cruise Ship']

    #Create empty list to which we will append countries with inconsistent coordinates


    #Create new df column and populate with some datetime value that we will replace
    data['virus_start_dt'] = pd.to_datetime(data['Date'].min())

    for i in data['Country'].unique():
        country_name = i
        if data['Confirmed'].loc[(data['Country']==country_name)].sum()== 0:
            data['virus_start_dt'].loc[(data['Country'] == country_name)] = np.NaN
        elif data['Confirmed'].loc[(data['Country']==country_name)].sum()> 0:
            virus_start_dt = data.loc[(data['Country']==country_name)&(data['Confirmed']>=1)].groupby('Country')['Date'].min()
            data['virus_start_dt'].loc[(data['Country'] == country_name)] = virus_start_dt[0]
    countries = []
    for i in data.groupby('Country').apply(lambda x: x['Country'].unique()):
        #Check and store countries that have more than one province in original dataset
        if data['Province/State'].loc[(data['Country']==i[0])].nunique()>1:
            countries.append(i[0])
        if i in countries:
            country_name = i[0]
            lat,long = do_geocode(country_name) #get new coordinates from geopy
            data['Lat'].loc[(data['Country'] == country_name)]=lat # update latitude
            data['Long'].loc[(data['Country'] == country_name)]=long #update longitude  
    
    # Aggregate clean data
    aggs = {'Confirmed': 'max', 'Deaths': 'max', 'Date':'max'}
    data = data.groupby(by=['Country', 'Lat', 'Long', 'virus_start_dt' ],  as_index=False).agg(aggs)
    print(data)
    data = data.rename(columns={'Date': 'last_upd'})
    data

    name = []
    for i in pycountry.countries:
        if hasattr (i, "name"):
            name.append(i.name)
        elif hasattr (i, "common_name"):
            name.append(i.common_name)
        elif hasattr (i, "official_name"):
            name.append(i.official_name)
    invalid = ([i for i in  data['Country'].unique() if(i not in name)])
    for i in invalid:
        pattern = re.compile(i)
        result = [j for j in name if pattern.match(j)]
        if result:
            data['Country'].loc[(data['Country'] == i)]=result[0]

    data['Country'].loc[(data['Country'] == "US")]='United States'
    #For each unique country in df get alpha 2 country code from pycountry 
    data['country_code'] = ""
    for i in data['Country']:
        data['country_code'].loc[(data['Country'] == i)] = get_country_code(i)

    #Hard-code country code for Congo
    data['country_code'].loc[(data['Country'] == "Congo (Brazzaville)")] = "CG"
    data['country_code'].loc[(data['Country'] == "Congo (Kinshasa)")] = "CD"

    #Read csv file with dw_pr countries and corresponding realms and codes into pandas df
    fullpath = os.path.join(base_dir, realms_file)
    realms = pd.read_csv(fullpath)

    #Grab two columns
    realms = realms[['ISO_ALPH2', 'WG_REGION']]

    #Rename column ISO_ALPH2 to match same column in data df
    realms = realms.rename(columns={'ISO_ALPH2': 'country_code'})

    #Merge two dfs on common column = country_code
    clean = pd.merge(data, realms, on='country_code', how='left')

    #Realm "NA" gets in interpreted as NaN, fix that
    clean['WG_REGION'].fillna("NA", inplace=True)

    #Create path to where output will be saved
    new_path = os.path.join(base_dir, new_file)

    #Return cleaned combined data
    return clean.to_csv(new_path, index = False)
    

In [69]:
base_dir = os.getcwd()
realms_file = "realms.csv"
new_file = "cleaned.csv"

clean_data(base_dir, realms_file, new_file)

                Country      Lat      Long virus_start_dt  Confirmed  Deaths  \
0           Afghanistan  33.0000   65.0000     2020-02-24      174.0     4.0   
1               Albania  41.1533   20.1683     2020-03-09      243.0    15.0   
2               Algeria  28.0339    1.6596     2020-02-25      716.0    44.0   
3               Andorra  42.5063    1.5218     2020-03-02      376.0    12.0   
4                Angola -11.2027   17.8739     2020-03-20        7.0     2.0   
..                  ...      ...       ...            ...        ...     ...   
175           Venezuela   6.4238  -66.5897     2020-03-14      135.0     3.0   
176             Vietnam  16.0000  108.0000     2020-01-23      212.0     0.0   
177  West Bank and Gaza  31.9522   35.2332     2020-03-05      119.0     1.0   
178              Zambia -15.4167   28.2833     2020-03-18       35.0     0.0   
179            Zimbabwe -20.0000   30.0000     2020-03-20        8.0     1.0   

          Date  
0   2020-03-31  
1   2

In [71]:
base_dir = os.getcwd()
realms_file = "realms.csv"
new_file = "timeseries.csv"
url = "https://raw.githubusercontent.com/datasets/covid-19/master/data/time-series-19-covid-combined.csv"

# read data into pandas df and do some renaming&cleaning
data = pd.read_csv(url, error_bad_lines=False, parse_dates=['Date'])
data = data.rename(columns={'Country/Region': 'Country'})
data = data[data.Country != 'Cruise Ship']


#Create new df column and populate with some datetime value that we will replace
data['virus_start_dt'] = pd.to_datetime(data['Date'].min())

for i in data['Country'].unique():
    country_name = i
    if data['Confirmed'].loc[(data['Country']==country_name)].sum()== 0:
        data['virus_start_dt'].loc[(data['Country'] == country_name)] = np.NaN
    elif data['Confirmed'].loc[(data['Country']==country_name)].sum()> 0:
        virus_start_dt = data.loc[(data['Country']==country_name)&(data['Confirmed']>=1)].groupby('Country')['Date'].min()
        data['virus_start_dt'].loc[(data['Country'] == country_name)] = virus_start_dt[0]

#Create empty list to which we will append countries with inconsistent coordinates
countries = []
for i in data.groupby('Country').apply(lambda x: x['Country'].unique()):
    #Check and store countries that have more than one province in original dataset
    if data['Province/State'].loc[(data['Country']==i[0])].nunique()>1:
        countries.append(i[0])
    if i in countries:
        country_name = i[0]
        lat,long = do_geocode(country_name) #get new coordinates from geopy
        data['Lat'].loc[(data['Country'] == country_name)]=lat # update latitude
        data['Long'].loc[(data['Country'] == country_name)]=long #update longitude  
            
for i, j in data.iterrows():
    if j['virus_start_dt']> j['Date']:
        data.drop(i, inplace=True)


#Aggregate clean data by country  
aggs = {'Confirmed': 'sum', 'Deaths': 'sum', }
data = data.groupby(by=['Date', 'Country', 'Lat', 'Long', 'virus_start_dt' ],  as_index=False).agg(aggs)
data = data.rename(columns={'Date': 'last_upd'})
name = []

for i in pycountry.countries:
    if hasattr (i, "name"):
        name.append(i.name)
    elif hasattr (i, "common_name"):
        name.append(i.common_name)
    elif hasattr (i, "official_name"):
        name.append(i.official_name)
invalid = ([i for i in  data['Country'].unique() if(i not in name)])
for i in invalid:
    pattern = re.compile(i)
    result = [j for j in name if pattern.match(j)]
    if result:
        data['Country'].loc[(data['Country'] == i)]=result[0]

data['Country'].loc[(data['Country'] == "US")]='United States'
#For each unique country in df get alpha 2 country code from pycountry 
data['country_code'] = ""

for i in data['Country']:
    data['country_code'].loc[(data['Country'] == i)] = get_country_code(i)

#Hard-code country code for Congo
data['country_code'].loc[(data['Country'] == "Congo (Brazzaville)")] = "CG"
data['country_code'].loc[(data['Country'] == "Congo (Kinshasa)")] = "CD"

#Read csv file with dw_pr countries and corresponding realms and codes into pandas df
fullpath = os.path.join(base_dir, realms_file)
realms = pd.read_csv(fullpath)

#Grab two columns
realms = realms[['ISO_ALPH2', 'WG_REGION']]

#   #Rename column ISO_ALPH2 to match same column in data df
realms = realms.rename(columns={'ISO_ALPH2': 'country_code'})

#Merge two dfs on common column = country_code
clean = pd.merge(data, realms, on='country_code', how='left')

#Realm "NA" gets in interpreted as NaN, fix that
clean['WG_REGION'].fillna("NA", inplace=True)

#Create path to where output will be saved
new_path = os.path.join(base_dir, new_file)

#Return cleaned combined data
clean.to_csv(new_path, index = False)
    